# Introduction

We will be working on a $t\bar{t}$ cross section measurement analysis using opendata from the CMS experiment. The lepton+jets final state is chosen: $t\bar{t} \to bW^+bW^- \to bq\bar{q}bl^-\nu_l$, characterized by one lepton (here we look at electrons and muons only), significant missing transverse energy, and four jets, two of which are b-tagged. We will first start by inspecting one [root file](https://root.cern/manual/root_files/) from the simulated inclusive $t\bar{t}$ sample. [Coffea](https://coffea-hep.readthedocs.io/en/stable/) (Columnar Object Framework for Effective Analysis) is a python package used for this purpose. It uses uproot for reading root files, awkward arrays to represent events as columnar information. To read the CMS NanoAOD format, we use `NanoEventsFactory` and `NanoAODSchema` but other methods exist to read flat ntuples as well.

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
events = NanoEventsFactory.from_root(
{"https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19980_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1_00000_0000.root" : "Events"},
    schemaclass=NanoAODSchema,
    entry_start=0,
    entry_stop=1000,
).events()

Now that the file is open, you can inspect the arrays we will be using.

In [ ]:
import awkward as ak
ak.num(events.Electron, axis=1).compute()
ak.num(events.Muon, axis=-1).compute()
ak.num(events.Jet, axis=1).compute()

In [ ]:
events.Electron.fields
events.Muon.fields
events.Jet.fields

We can check the effect of applying quality criteria on the particle collections:

In [ ]:
import numpy as np
selected_electrons = events.Electron[(events.Electron.pt > 30) & (np.abs(events.Electron.eta) < 2.5) & (events.Electron.isTight == True)]
selected_muons = events.Muon[(events.Muon.pt > 30) & (np.abs(events.Muon.eta) < 2.4) & (events.Muon.mediumId == True)]
selected_jets = events.Jet[(events.Jet.pt > 25) & (np.abs(events.Jet.eta) < 2.4)]

In [ ]:
events.Jet.pt[0].compute()

In [ ]:
selected_jets.pt[0].compute()

Next we will be looking at an event-level variable. After selecting the objects we require events to be selected by `event_filters` if they have at least four jets, two of them must be b-tagged. B-tagging discriminator scores (CSVV2) are stored in the root file per jet and higher values indicate higher probability that the jet originates from a b quark. In the CMS experiment we generally use a score threshold or working point which has 80% signal efficiency and 1% mis-tagging rate for light quarks. We want to then calculate the tri-jet invariant mass from the fully hadronic top decay: $t\to bW^+ \to bq\bar{q}$. Note that the awakward array feature `combinations` makes it easy to implement the combinatorics involved in finding multiple tri-jet candidates which are then sorted based on b-tagging and $p_T$. Here is where the true power of columnar analysis and the wonderful python tools that are being developed become really evident. 

In [ ]:
def calculate_trijet_mass(events):
    # pT > 30 GeV for leptons, > 25 GeV for jets
    selected_electrons = events.Electron[(events.Electron.pt > 30) & (np.abs(events.Electron.eta) < 2.5) & (events.Electron.isTight == True)]
    selected_muons = events.Muon[(events.Muon.pt > 30) & (np.abs(events.Muon.eta) < 2.4) & (events.Muon.mediumId == True)]
    selected_jets = events.Jet[(events.Jet.pt > 25) & (np.abs(events.Jet.eta) < 2.4)]

    # single lepton requirement
    event_filters = ((ak.count(selected_electrons.pt, axis=1) + ak.count(selected_muons.pt, axis=1)) == 1)
    # at least four jets
    event_filters = event_filters & (ak.count(selected_jets.pt, axis=1) >= 4)
    # at least two b-tagged jets ("tag" means score above threshold)
    B_TAG_THRESHOLD = 0.8
    event_filters = event_filters & (ak.sum(selected_jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) >= 2)

    # apply filters
    selected_jets = selected_jets[event_filters]

    trijet = ak.combinations(selected_jets, 3, fields=["j1", "j2", "j3"])  # trijet candidate
    trijet["p4"] = trijet.j1 + trijet.j2 + trijet.j3  # four-momentum of tri-jet system

    trijet["max_btag"] = np.maximum(trijet.j1.btagCSVV2, np.maximum(trijet.j2.btagCSVV2, trijet.j3.btagCSVV2))
    trijet = trijet[trijet.max_btag > B_TAG_THRESHOLD]  # at least one-btag in trijet candidates
    # pick trijet candidate with largest pT and calculate mass of system
    trijet_mass = trijet["p4"][ak.argmax(trijet.p4.pt, axis=1, keepdims=True)].mass
    return ak.flatten(trijet_mass)


In [ ]:
calculate_trijet_mass(events).compute()

Coffea hist subpackage facilitates plotting by creating Hist objects. In physics analyses, we use histograms to visualize the data. We can’t plot every value of trijet mass, so we divide the range of masses into n bins across some reasonable range. Thus, we need to define the mapping for our reduction; defining the number of bins and the range is sufficient for this. This is called a Regular axis in the hist.Hist package, we are using 25 bins between values of 50 and 550 GeV.

In [ ]:
import hist
histogram = hist.Hist.new.Reg(25, 50, 550, name="observable", label="observable [GeV]").StrCat(["4j1b", "4j2b"], name="region", label="Region").StrCat([], name="process", label="Process", growth=True).StrCat([], name="variation", label="Systematic variation", growth=True).Weight()
histogram.fill(observable=calculate_trijet_mass(events).compute(), region="4j2b", process="ttbar", variation="nominal", weight=1)

In [ ]:
import matplotlib.pyplot as plt
histogram[:,"4j2b","ttbar","nominal"].plot(histtype="fill", linewidth=1, edgecolor="grey", label='ttbar')
plt.legend()
plt.title(">= 4 jets, >= 2 b-tags")
plt.xlabel("$m_{bjj}$ [Gev]");
plt.show()

# Coffea executor

We would now like to scale this analysis up to far larger datasets for a practical analysis scenario. The first expansion we can do to our analysis is to consider running it over more datasets, which include all of our data, our background, and the signal. Additionally, we would like to show to how to estimate a few sources of systematic uncertainty, and for that we will be using, in some cases, additional datasets. These systematics datasets are generaly variations of the nominal ones.

To expand our analysis, we will use coffea Processors. Processors are coffea’s way of encapsulating an analysis in a way that is deployment-neutral. Once you have a Coffea analysis, you can throw it into a processor and use any of a variety of executors (e.g. Dask, Parsl, Spark) to chunk it up and run it across distributed workers. This makes scale-out simple and dynamic for users. Unfortunately, we don’t have the time to do such a demostration, but we will run it locally, with our vanilla coffea executor.

In [ ]:
from coffea.processor import ProcessorABC
import hist
import awkward as ak
import numpy as np
class TtbarAnalysis(ProcessorABC):
    def __init__(self):
        num_bins = 25
        bin_low = 50
        bin_high = 550
        name = "observable"
        label = "observable [GeV]"
        #https://hist.readthedocs.io/en/latest/user-guide/quickstart.html
        #StrCat = StrCategory
        #https://hist.readthedocs.io/en/latest/banner_slides.html?highlight=StrCategory#many-axis-types
        self.hist = (
            hist.Hist.new.Reg(num_bins, bin_low, bin_high, name=name, label=label)
            .StrCat(["4j1b", "4j2b"], name="region", label="Region")
            .StrCat([], name="process", label="Process", growth=True)
            .StrCat([], name="variation", label="Systematic variation", growth=True)
            .Weight()
        )

    def process(self, events):
        hist_4j1b = self.hist.copy()
        hist_4j2b = self.hist.copy()
        hist_dict = {"4j1b": hist_4j1b, "4j2b": hist_4j2b}
        process = events.metadata["process"]  # "ttbar" etc.
        variation = events.metadata["variation"]  # "nominal", "scaledown", etc.

        # normalization for MC
        x_sec = events.metadata["xsec"]
        nevts_total = events.metadata["nevts"]
        lumi = 2256.38
        if process != "data":
            xsec_weight = x_sec * lumi / nevts_total
        else:
            xsec_weight = 1

        #object selection
        selected_electrons = events.Electron[(events.Electron.pt > 30) & (abs(events.Electron.eta)<2.1) & (events.Electron.isTight == True)]
        selected_muons = events.Muon[(events.Muon.pt > 30) & (abs(events.Muon.eta)<2.1) & (events.Muon.mediumId == True)]
        selected_jets = events.Jet[(events.Jet.pt > 30) & (abs(events.Jet.eta)<2.4)]

        # single lepton requirement
        event_filters = ((ak.count(selected_electrons.pt, axis=1) + ak.count(selected_muons.pt, axis=1)) == 1)

        # at least four jets
        event_filters = event_filters & (ak.count(selected_jets.pt, axis=1) >= 4)

        # at least one b-tagged jet ("tag" means score above threshold)
        B_TAG_THRESHOLD = 0.8
        event_filters = event_filters & (ak.sum(selected_jets.btagCSVV2 >= B_TAG_THRESHOLD, axis=1) >= 1)

        # apply event filters
        selected_events = events[event_filters]
        selected_electrons = selected_electrons[event_filters]
        selected_muons = selected_muons[event_filters]
        selected_jets = selected_jets[event_filters]

        for region in ["4j1b", "4j2b"]:
            # further filtering: 4j1b CR with single b-tag, 4j2b SR with two or more tags
            if region == "4j1b":
                region_filter = ak.sum(selected_jets.btagCSVV2 >= B_TAG_THRESHOLD, axis=1) == 1
                selected_jets_region = selected_jets[region_filter]
                # use HT (scalar sum of jet pT) as observable
                observable = ak.sum(selected_jets_region.pt, axis=-1).compute()
                hist_4j1b.fill(observable=observable, region=region, process=process, variation=variation, weight=xsec_weight)

            elif region == "4j2b":
                region_filter = ak.sum(selected_jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) >= 2
                selected_jets_region = selected_jets[region_filter]

                # reconstruct hadronic top as bjj system with largest pT
                # the jet energy scale / resolution effect is not propagated to this observable at the moment
                trijet = ak.combinations(selected_jets_region, 3, fields=["j1", "j2", "j3"])  # trijet candidates
                trijet["p4"] = trijet.j1 + trijet.j2 + trijet.j3  # calculate four-momentum of tri-jet system
                trijet["max_btag"] = np.maximum(trijet.j1.btagCSVV2, np.maximum(trijet.j2.btagCSVV2, trijet.j3.btagCSVV2))
                trijet = trijet[trijet.max_btag > B_TAG_THRESHOLD]  # require at least one-btag in trijet candidates
                # pick trijet candidate with largest pT and calculate mass of system
                trijet_mass = trijet["p4"][ak.argmax(trijet.p4.pt, axis=1, keepdims=True)].mass
                observable = ak.flatten(trijet_mass).compute()
                hist_4j2b.fill(observable=observable, region=region, process=process, variation=variation, weight=xsec_weight)

        output = {"hist": hist_dict}

        return output

    def postprocess(self, accumulator):
        pass


The datasets over which we are running are still quite large. So running on the whole dataset in a single computer is not very efficient. Here is where coffea really performs, because you can ship it to different worker nodes using some different executors. In this example we will use `dask` but on a single file.

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

events = NanoEventsFactory.from_root(
    {"https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19980_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1_00000_0000.root": "Events"},
    metadata={"process": "ttbar", "variation" : "nominal", "xsec" : 833, "nevts" : 1334428},
    schemaclass=NanoAODSchema,
).events()

In [ ]:
import dask
p = TtbarAnalysis()
out = p.process(events)
(computed,) = dask.compute(out)
print(computed)

You can add the control region histogram here for visualization.

In [ ]:
import matplotlib.pyplot as plt

hist_4j1b = computed['hist']['4j1b']
print(hist_4j1b)

hist_4j1b_nominal = hist_4j1b[
    dict(region='4j1b', process='ttbar', variation='nominal')
]

fig, ax = plt.subplots()
hist_4j1b_nominal.plot1d(ax=ax)
plt.show()

# Running over a list of samples

In an analysis we will have a list of files we need to analyze, for this purpose we will use the json format. Inspect the `nanoaod_inputs.json` file content which lists several samples along with some metadata. We want better statistics of the Monte Carlo samples to reduce uncertainty from statistical fluctuations. In addition, we use these samples for background nomalization, various efficiency measurements, kinematics studies, machine-learning model training etc. Now we will be using 20 files from the ttbar sample to obatin the same histograms. 

In [ ]:
import json
with open("nanoaod_inputs.json") as f:
    file_info = json.load(f)

files = {}

tot_events = file_info["ttbar"]["nominal"]["nevts_total"]
file_list = file_info["ttbar"]["nominal"]["files"]
file_paths = [f["path"] for f in file_list]

nevts_total = sum([f["nevts"] for f in file_list[:20]])

# Create { "path/to/file.root": "Events" } for each file
files_dict = {path: "Events" for path in file_paths[:20]}

print(len(files_dict))
print(nevts_total)

In [ ]:
events_nom = NanoEventsFactory.from_root(
    files_dict,
    metadata={"process": "ttbar", "variation" : "nominal", "xsec" : 833, "nevts" : nevts_total},
    schemaclass=NanoAODSchema,
).events()

In [ ]:
import time

t0 = time.monotonic()
out_nom = p.process(events_nom)
(computed_nom,) = dask.compute(out_nom)
exec_time = time.monotonic() - t0

print(computed_nom)
print(f"\nexecution took {exec_time:.2f} seconds")

We are roughly analyzing 130k events per second. From here you can add the functionality to process files from each sample in the json and create a collection of histograms for further analaysis. [Here](https://cernbox.cern.ch/index.php/s/SPcoOLArZCFFupN) you can download a pre-made `histograms.root` file. It contains all the histograms produced by the analysis processor and was obtained after 4 hours of running on a single computer. Of course, if you have some available space, you could download the files (the size is not terribly large, maybe around 60GB or so), and run much faster without replying on remote server. Use the provided plotting script to visualize the results: 

In [ ]:
import uproot
import hist
import mplhep as hep
import matplotlib.pyplot as plt

file = uproot.open("histograms.root")
file.classnames()
data = file['4j2b_data'].to_hist()
ttbar = file['4j2b_ttbar'].to_hist()
single_atop = file['4j2b_single_atop_t_chan'].to_hist()
single_top = file['4j2b_single_top_t_chan'].to_hist()
single_tW = file['4j2b_single_top_tW'].to_hist()
wjets = file['4j2b_wjets'].to_hist()
bklabels = ["ttbar","sigle_atop","single_top","single_tW","wjets"]

hep.style.use("CMS")
hep.cms.label("open data",data=True, lumi=2.26, year=2015)
hep.histplot(data,histtype="errorbar", color='k', capsize=4, label="Data")
hep.histplot([ttbar,single_atop,single_top,single_tW,wjets],stack=True, histtype='fill', label=bklabels, sort='yield')
plt.legend(frameon=False)
plt.xlabel("$m_{bjj}$ [Gev]");
plt.savefig('finalplot.png')
plt.show()